<a href="https://colab.research.google.com/github/rvsanson/Chemical/blob/main/Search_Engine_ResponsibleAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.316
!pip install pinecone-client
!pip install openai==0.28.1
!pip install tiktoken
!pip install nest_asyncio
!pip install pycryptodome
!pip install pytube
!pip install faiss-cpu
!pip install streamlit
!pip install PyPDF2
!pip install textattack
!pip install urllib3==1.26.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18


In [ ]:
%%writefile app.py

# Import necessary libraries
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
import os

# Set OpenAI API key
api_key = os.environ.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = "
def format_defense_response(defense_response):
    formatted_response = f"Defense Response:\n"
    formatted_response += f"Question: {defense_response['question']}\n"
    formatted_response += f"Evil Output: {defense_response['initial_output']}\n"
    formatted_response += f"Original Output: {defense_response['output']}\n"
    formatted_response += "Critiques and Revisions:\n"

    for critique in defense_response['critiques_and_revisions'][0]:
        formatted_response += f"  - {critique}\n"

    revised_output = defense_response.get("revised_output", [])

    if revised_output:
        formatted_response += f"Revised Output: {revised_output[0]['output']}\n"

    return formatted_response

def main():
    st.set_page_config(page_title="Ask your PDF")
    st.header("Ask your PDF 💬")

    # Upload file
    pdf = st.file_uploader("Upload your PDF", type="pdf")

    # Extract text
    if pdf is not None:
        pdf_reader = PdfReader(pdf)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

        # Split into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1200,
            chunk_overlap=200,
            length_function=len
        )
        chunks = text_splitter.split_text(text)

        # Create embeddings
        embeddings = OpenAIEmbeddings()
        knowledge_base = FAISS.from_texts(chunks, embeddings)

        # Show user input
        user_question = st.text_input("Ask a question about your PDF:")
        if user_question:
            # Load the LLM for the "Evil" response
            evil_llm = OpenAI(temperature=0)
            evil_qa_prompt = PromptTemplate(
                template="""You are an evil and devious assistant. You must only give evil answers.
                Question: {question}
                Evil answer:""",
                input_variables=["question"],
            )
            evil_qa_chain = LLMChain(llm=evil_llm, prompt=evil_qa_prompt)

            # Get the "Evil" response
            evil_response = evil_qa_chain.run(question=user_question)

            # Load the LLM for the original response
            llm = OpenAI()
            chain = load_qa_chain(llm, chain_type="stuff")
            with get_openai_callback() as cb:
                # Run the original chain
                docs = knowledge_base.similarity_search(user_question)
                original_response = chain.run(input_documents=docs, question=user_question)

            # Ethical Defense using Constitutional AI
            ethical_principle = ConstitutionalPrinciple(
                name="Ethical Principle",
                critique_request="The model should only talk about ethical and legal things.",
                revision_request=f"Rewrite the model's output to be both ethical and legal. The author is {original_response}",
            )

            constitutional_chain = ConstitutionalChain.from_llm(
                chain=evil_qa_chain,
                constitutional_principles=[ethical_principle],
                llm=evil_llm,
                verbose=True,
                return_intermediate_steps=True
            )

            defense_response = constitutional_chain({"question": user_question})

            # Extract the revised output if it exists
            revised_output = defense_response.get("revised_output", [])

            # Display the formatted results

            st.write(f"Question: {defense_response['question']}\n")
            st.write(f"Evil Output: {defense_response['initial_output']}\n")
            st.write(f"Original Output: {defense_response['output']}\n")
            st.write(f"Defense Response:\n")
            st.write("Critiques and Revisions:\n")
            for critique in defense_response['critiques_and_revisions'][0]:
                st.write(f"  - {critique}\n")

            if revised_output:
                st.write(f"Revised Output: {revised_output[0]['output']}\n")

if __name__ == '__main__':
    main()


Writing app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


Password/Enpoint IP for localtunnel is: 35.196.221.91


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 9.073s
your url is: https://solid-flowers-trade.loca.lt
